In [2]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [5]:
pd.DataFrame({'title':'_', 'author':'_', 'genre_1':'_', 'genre_2':'_', 'rank':0, 'rate':0, 'count':0}, index=[0])

,title,author,genre_1,genre_2,rank,rate,count
0,_,_,_,_,0,0,0


In [125]:
df = pd.read_csv('C:/Users/Playdata/playdata_wj/Project/gyobo_소설시희곡.csv')
print(df.duplicated(['title','author']).sum())
df.drop_duplicates(subset=['title','author','rate'], ignore_index=True, inplace=True)
print(df.duplicated(['title','author']).sum())

0
0


In [133]:
df[df['genre_2'].str.contains('정신건강')]

,title,author,genre_1,genre_2,rate,female_10,male_10,female_20,male_20,female_30,male_30,female_40,male_40,female_50,male_50,female_60,male_60
71,이시형 박사의 둔하게 삽시다,이시형,"에세이, 건강/취미, 에세이, 에세이","명사에세이, 정신건강, 심리치유 에세이, 한국에세이",8.8,1.7,0.2,13.4,2.8,21.3,7.1,19.8,10.0,10.2,9.6,1.7,2.2
119,몸의 심리학,이병창,"건강/취미, 건강/취미, 인문학","건강정보, 정신건강, 심리학/정신분석학",10.0,0.0,0.0,1.7,1.3,13.8,6.3,27.6,14.6,7.5,17.2,2.5,7.5
275,5분의 기적 EFT,최인원,"건강/취미, 건강/취미, 건강/취미, 건강/취미, 인문학, 자기계발, 자기계발","대체의학, 건강정보, 정신건강, 한의학, 심리학/정신분석학, 인간관계, 힐링",7.8,0.3,0.0,7.5,3.7,14.3,7.5,25.3,11.5,13.9,9.1,1.8,5.1
286,네 가지 질문,바이런 케이티,"건강/취미, 에세이, 에세이, 자기계발, 자기계발","정신건강, 명상에세이, 외국에세이, 인간관계, 힐링",9.6,0.2,0.2,6.0,2.5,15.8,4.1,26.3,9.7,17.3,10.3,3.3,4.3
406,진실이 치유한다,데보라 킹,"에세이, 건강/취미, 건강/취미, 건강/취미, 에세이, 자기계발, 자기계발, 자기계발","외국에세이, 건강정보, 대체의학, 정신건강, 명상에세이, 여성의 자기계발, 힐링, 힐링",9.4,0.2,0.2,5.8,1.8,19.9,4.3,30.5,10.7,15.8,6.0,1.5,3.4
455,당신이 플라시보다,조 디스펜자,"인문학, 건강/취미, 자기계발","심리학/정신분석학, 정신건강, 힐링",8.8,0.1,0.1,5.3,3.4,15.8,7.8,23.0,10.4,16.5,10.6,2.6,4.4
524,고엔카의 위빳사나 명상,S.N. 고엔카,"인문학, 건강/취미, 종교/역학, 종교/역학","교양 인문학, 정신건강, 명상/선, 불교",8.7,0.1,0.2,6.2,6.2,17.6,9.4,19.8,11.7,10.6,10.9,2.4,4.7
546,고엔카의 위빳사나 10일 코스,S.N. 고엔카,"인문학, 건강/취미, 종교/역학, 종교/역학","교양 인문학, 정신건강, 명상/선, 불교",8.0,0.1,0.0,5.1,5.1,14.5,9.5,16.6,13.0,11.4,14.2,3.5,7.0
550,"알렉산더 테크닉, 내 몸의 사용법",프레더릭 알렉산더,"건강/취미, 건강/취미, 건강/취미","건강정보, 정신건강, 질병치료와 예방",8.0,0.3,0.0,9.1,9.1,15.4,12.2,21.6,11.4,9.3,7.0,1.4,3.2
806,루이스 헤이의 나를 치유하는 생각,루이스 L. 헤이,"자기계발, 건강/취미","힐링, 정신건강",9.3,0.3,0.3,8.0,2.1,25.9,3.4,33.2,6.0,14.5,3.8,1.5,1.0


In [160]:
class recommBook():
    def __init__(self, gender, age, genres):
        self.ga = 2*(age//10)-2 if gen=='female' else 2*(age//10)-1
        # self.genres = {g1:{'df':df, 'g2_11':df_11, 'g2_12':df_12}}
        self.genres = {g1:{g2:None for g2 in g2s} for g1,g2s in genres.items()}
        for g1 in self.genres.keys():
            self.genres[g1]['df'] = self.call_df(g1)
        print('user info: {}, {}, {}, {}'.format(gender, age, genres))
        
    def call_df(sefl, g1):
        df = pd.read_csv('C:/Users/Playdata/playdata_wj/Project/gyobo_{}.csv'.format(g1))
        print(df.duplicated(['title','author']).sum(), end='->')
        df.drop_duplicates(subset=['title','author'], ignore_index=True, inplace=True)
        print(df.duplicated(['title','author']).sum())
        return df
    
    def recomm1(self, g1, g2, n=240, check=True):
        df = self.genres[g1]['df'].copy()
        
        try:
            idx = df[df['genre_2']==g2].index.values[0]
        except:
            check = False
            df = pd.concat([df, pd.DataFrame({'genre_2':[g2]})], ignore_index=True)
            idx = df[df['genre_2']==g2].index.values

        cvect = CountVectorizer(ngram_range=(1,3))
        cvect_g2 = cvect.fit_transform(df['genre_2'])
        csim_g2 = cosine_similarity(cvect_g2, cvect_g2).argsort()[:,::-1]
        
        sim_idx = csim_g2[idx, :n].reshape(-1)
        sim_idx = sim_idx if check else sim_idx[sim_idx != idx]
        self.genres[g1][g2] = df.iloc[sim_idx].sort_values(['rank', 'rate'], ascending=[False, False])
        return self.genres[g1][g2]
    
    def recomm1_result(self):
        select_dfs = []
        g1_n, g1_c = 240//len(self.genres.keys(), 12//len(self.genres.keys())
        for g1, g2s in self.genres.keys():
            g2_n, g1_c = g1_n//(len(g2s)-1), g2_c//(len(g2s)-1)
            for g2 in g2s.keys():
                if g2=='df': continue
                select_dfs.append(self.recomm1(g1, g2, g2_n).iloc[g2_n:]
        return pd.concat(select_dfs, axis=1)
        
#     def recomm_GenAge(self, df, n=15):
#         target = np.array([.0 for i in range(12)])
#         target[self.ga] = 99.9
#         target = target.reshape(-1,12)
#         concat_df = pd.concat([df, pd.DataFrame(target, columns=df.columns[6:].values)], ignore_index=True)

#         csim_ga = cosine_similarity(concat_df.iloc[:,6:], concat_df.iloc[:,6:]).argsort()[:,::-1]

#         idx = concat_df.index.values[-1]
#         sim_idx = csim_ga[idx, :n].reshape(-1)
#         sim_idx = sim_idx[sim_idx != idx]
#         return df.iloc[sim_idx].sort_values(df.columns[5+self.ga], ascending=False)
    
#     def recomm_g2(self, df, title, n=15):
#         cvect = CountVectorizer(ngram_range=(1,3))
#         cvect_g2 = cvect.fit_transform(df['genre_2'])
#         csim_g2 = cosine_similarity(cvect_g2, cvect_g2).argsort()[:,::-1]

#         idx = df[df['title'] == title].index.values
#         sim_idx = csim_g2[idx, :n].reshape(-1)
#         sim_idx = sim_idx[sim_idx != idx]
#         return df.iloc[sim_idx].sort_values(df.columns[5+self.ga], ascending=False)

In [163]:
gender, age = 'male', 20
genres = {} # g1:[g2_1, g2_2]
myrecomm = recommBook(gender, age, genres)
myrecomm.recomm1(g1_1, g2_11).iloc[2:]
myrecomm.recomm1(g1_1, g2_12).iloc[2:]
myrecomm.recomm1(g1_2, g2_21).iloc[2:]
myrecomm.recomm1(g1_2, g2_22).iloc[2:]
myrecomm.recomm1(g1_3, g2_31).iloc[2:]
myrecomm.recomm1(g1_3, g2_32).iloc[2:]
# myrecomm.recomm_g2(df, '마션')

user info: male, 20, 소설시희곡, 판타지/환상문학
3->0


,title,author,genre_1,genre_2,rate,female_10,male_10,female_20,male_20,female_30,male_30,female_40,male_40,female_50,male_50,female_60,male_60
1212,나 혼자만 레벨업 7,추공,소설/시/희곡,판타지/환상문학,0.0,1.8,3.5,6.6,25.4,6.7,6.0,28.3,9.8,5.9,4.9,0.6,0.5
1214,나 혼자만 레벨업 6,추공,소설/시/희곡,판타지/환상문학,10.0,1.9,3.4,6.3,24.9,6.8,6.0,29.2,9.9,5.8,4.7,0.5,0.4
1211,나 혼자만 레벨업 8,추공,소설/시/희곡,판타지/환상문학,8.0,2.0,3.6,6.5,24.9,6.7,6.0,28.4,10.2,5.8,4.9,0.5,0.5
869,룬의 아이들 - 블러디드 2,전민희,"소설/시/희곡, 소설/시/희곡","판타지/환상문학, 한국소설",10.0,2.1,0.8,38.1,22.4,14.8,10.9,4.9,1.7,2.5,1.1,0.4,0.2
1061,룬의 아이들 - 블러디드 3,전민희,"소설/시/희곡, 소설/시/희곡","판타지/환상문학, 한국소설",8.8,1.5,0.6,35.6,19.3,19.5,13.0,5.0,1.5,2.6,1.0,0.2,0.2
1246,룬의 아이들 - 블러디드 4,전민희,"소설/시/희곡, 소설/시/희곡","판타지/환상문학, 한국소설",9.8,1.4,0.4,30.7,17.9,22.3,15.3,5.6,2.0,2.7,1.1,0.2,0.3
884,룬의 아이들 - 윈터러 세트 - 전7권,전민희,"소설/시/희곡, 소설/시/희곡","판타지/환상문학, 한국소설",9.6,2.6,0.8,37.8,17.5,15.0,9.0,8.9,3.3,2.7,2.0,0.2,0.2
680,오버 더 초이스 세트 - 전2권,이영도,"소설/시/희곡, 소설/시/희곡","판타지/환상문학, 한국소설",9.3,0.7,1.1,14.5,15.5,22.3,26.2,8.6,7.6,1.8,1.1,0.3,0.3
1091,룬의 아이들 - 데모닉 1~9 세트 - 전9권,전민희,"소설/시/희곡, 소설/시/희곡","판타지/환상문학, 한국소설",10.0,1.4,0.6,38.6,14.4,19.0,9.8,7.6,3.3,2.9,2.1,0.1,0.2
1077,시하와 칸타의 장,이영도,"소설/시/희곡, 소설/시/희곡","판타지/환상문학, 한국소설",8.2,0.6,0.4,13.2,13.4,22.3,25.9,9.5,10.0,2.4,1.4,0.4,0.4
